In [15]:
import socket

message_end_token = b"000000"

def receive_data(address=""):
    sock = socket.socket()
    sock.bind((address, 5007))
    
    chunks = []
    while True:
        chunk = sock.recv(4096)
        if chunk.endswith(message_end_token):
            break
        if chunk == b'':
            raise "socket connection broken"
    
    bytestring = b''.join(chunk)[:len(message_end_token)]
    return float(bytestring)

print(receive_data())


OSError: [Errno 107] Transport endpoint is not connected

In [ ]:
import socket
import struct

message_end_token = b"000000"

def receive_temperature_data(address="", port = 5015, timeout = 15*60):
    sock = socket.socket()
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.settimeout(timeout)
    sock.bind((address, port))
    
    sock.listen(2)
    conn, addr = sock.accept()
    
    try: 
        data = conn.recv(64)
    except Exception as e:
        sock.close()
        raise e
        
    sock.close()
    
    temp, = struct.unpack("<f", data) # convert the bytes into a float (little endian)
    
    assert temp > 0.0
    assert temp < 300.0
    return temp


def stream_temperature_from(address, port = 5015, interval = 15*60):
    temperature = None
    while True:
        try:
            temperature = receive_temperature_data(address, port, interval)
        except socket.error as e:
            # Presuming this is timeout
            # Write the data somewhere
            print(e)
            print(temperature)


stream_temperature_from("", interval = 1)

[Errno 98] Address already in use
None


In [ ]:
    sock.listen(2)
    conn, addr = sock.accept()
    while True:
        chunk = sock.recv(4096)
        if chunk.endswith(message_end_token):
            break
        if chunk == b'':
            raise "socket connection broken"
    data = b''.join(chunk)[:len(message_end_token)]
    
    sock.close()